In [670]:
%matplotlib inline
import pandas as pd
import numpy as np
import single_df_curve.single_df_curve as sc
import matplotlib.pyplot as plt
import os

In [605]:
IR_base_date = '20180731'
root_path = '/Users/susynishida/Desktop/dev/Python/Finance/input/'
IR_input = '/IR.csv'
df_IR_180731 = pd.read_csv(root_path + IR_base_date + IR_input)
DF_usd_obj_180731 = sc.discount_factor(df_IR_180731, 'USD')

In [606]:
IR_base_date = '20180928'
root_path = '/Users/susynishida/Desktop/dev/Python/Finance/input/'
IR_input = '/IR.csv'
df_IR_180928 = pd.read_csv(root_path + IR_base_date + IR_input)
DF_usd_obj_180928 = sc.discount_factor(df_IR_180928, 'USD')

In [610]:
DF_usd_obj_180731.get_array_DF('2018/10/10', '2018/10/15', 'cubic', 'cubic')

[('2018/10/10', 0.9958857266729872),
 ('2018/10/11', 0.9958132634994473),
 ('2018/10/12', 0.9957404438448808),
 ('2018/10/13', 0.9956672869007793),
 ('2018/10/14', 0.9955938118586336),
 ('2018/10/15', 0.9955200379099348)]

In [611]:
DF_usd_obj_180928.get_array_DF('2018/10/10', '2018/10/15', 'cubic', 'cubic')

[('2018/10/10', 0.9995575994574869),
 ('2018/10/11', 0.9994955567737117),
 ('2018/10/12', 0.999434104408596),
 ('2018/10/13', 0.9993732030737402),
 ('2018/10/14', 0.9993128134807455),
 ('2018/10/15', 0.9992528963412125)]

In [591]:
[j for i,j in DF_usd_obj.get_array_DF('2018/9/2', '2018/9/6', 'cubic', 'cubic')]

[0.9983264164458074,
 0.9982685198900594,
 0.998210393615349,
 0.9981520234270751,
 0.9980934001493967]

In [1010]:
class swap_pricer():
    def __init__(self, str_base_date, str_start_date, str_end_date, fixed_rate, int_notional, str_ccy, str_frequency_fixed_side, str_frequency_floot_side, str_dcc_fixed_side, str_dcc_floot_side, is_fixed_pay, df_IR):
        self._fixed_rate = fixed_rate
        self._dt_obj_base_date = str_to_dt_obj(str_base_date)
        self._boolean_fixed_side = is_fixed_pay
        self._notional = int_notional
        self._str_ccy = str_ccy
        self._dcc_fixed_side = float(str_dcc_fixed_side)
        self._dcc_floot_side = float(str_dcc_floot_side)
        self._DF_obj =  sc.discount_factor(df_IR, str_ccy)
        self._CF_df_fixed_side = get_CF_df_from_base_date(str_base_date, str_start_date, str_end_date, str_frequency_fixed_side, is_floot=False)
        self._CF_df_floot_side = get_CF_df_from_base_date(str_base_date, str_start_date, str_end_date, str_frequency_floot_side, is_floot=True) 
        self._dcf_list_fixed_side = self._create_list_dcf(self._CF_df_fixed_side, str_dcc_fixed_side)
        self._dcf_list_floot_side = self._create_list_dcf(self._CF_df_floot_side, str_dcc_floot_side)
     
    
    def get_DF(self, str_target_date):
        return self._DF_obj.get_DF(str_target_date, 'cubic', 'cubic')
    
    def get_CF_df_fixed_side(self):
        return self._CF_df_fixed_side 
    
    def get_CF_df_floot_side(self):
        return self._CF_df_floot_side 
    
    def get_dcf_list(self):
        return self._dcf_list_floot_side
    
    def _create_list_dcf(self, CF_df, str_dcc):
        len_CF_df = len(CF_df)
        dcf_list = []
        for i in range(len_CF_df):
            dcf = (str_to_dt_obj(CF_df.iloc[i]['end_date']) - str_to_dt_obj(CF_df.iloc[i]['start_date'])).days / float(str_dcc)
            dcf_list.append(dcf)
        return dcf_list
    
    def calc_pv_fix_side(self):
        len_CF = len(self._CF_df_fixed_side)
        pv = 0
        str_ini_sdate = self._CF_df_fixed_side.iloc[0]['start_date']
        str_ini_edate = self._CF_df_fixed_side.iloc[0]['end_date']
        if (str_to_dt_obj(str_ini_sdate) <= self._dt_obj_base_date <= str_to_dt_obj(str_ini_edate)):
            dcf_initial = (str_to_dt_obj(str_ini_edate) - self._dt_obj_base_date).days / self._dcc_fixed_side
            pv = self._fixed_rate * dcf_initial * self._DF_obj.get_DF(self._CF_df_fixed_side.iloc[0]['start_date'], 'cubic', 'cubic') 
            for i in range(len_CF - 1):
                pv = pv + self._fixed_rate * self._DF_obj.get_DF(self._CF_df_fixed_side.iloc[i]['end_date'], 'cubic', 'cubic') * self._dcf_list_fixed_side[i]
        else:
            for i in range(len_CF):
                pv = pv + self._fixed_rate * self._DF_obj.get_DF(self._CF_df_fixed_side.iloc[i]['end_date'], 'cubic', 'cubic') * self._dcf_list_fixed_side[i]
        return self._notional * pv
    
    def calc_pv_floot_side(self):
        len_CF = len(self._CF_df_floot_side)
        pv = 0
        forward_libor_list = self.generate_forward_libor()
        index_for_fixing_rate = self.get_index_for_fixing_rate()
        if (index_for_fixing_rate > 0):
            fixing_rate = df_fixing_rate.iloc[index_for_fixing_rate]['fixing_rate']
        # considering to aqure CF_date from CF.csv or from CF_after_trimming. In the case where end date is not difference from next CF start date,
        # we need to use CF.csv
            dcf_initial = (str_to_dt_obj(self._CF_df_floot_side.iloc[0]['start_date']) - self._dt_obj_base_date).days / self._dcc_floot_side
            pv = fixing_rate * dcf_initial * self._DF_obj.get_DF(self._CF_df_floot_side.iloc[0]['start_date'], 'cubic', 'cubic') 
            for i in range(len_CF - 1):
                pv = pv + forward_libor_list[i] * self._DF_obj.get_DF(self._CF_df_floot_side.iloc[i]['end_date'], 'cubic', 'cubic') * self._dcf_list_floot_side[i]
        else:
            for i in range(len_CF):
                pv = pv + forward_libor_list[i] * self._DF_obj.get_DF(self._CF_df_floot_side.iloc[i]['end_date'], 'cubic', 'cubic') * self._dcf_list_floot_side[i]
        return self._notional * pv
    
    def get_index_for_fixing_rate(self):
        df_fixing_rate = pd.read_csv('CF.csv').loc[:, 'fixing_date(floot)':'fixing_rate'] # to do: chnage 'CF.csv'
        index_for_fixing_rate = self._CF_df_floot_side.index[0] - 1
        return index_for_fixing_rate
    
    def generate_forward_libor(self):
        CF_df = self._CF_df_floot_side
        len_CF = len(CF_df)
        forward_libor_list = []
        dcf_list = self._dcf_list_floot_side
        for i in range(len_CF):
            DF_start_date = self._DF_obj.get_DF(self._CF_df_floot_side.iloc[i]['start_date'], 'cubic', 'cubic')
            DF_end_date = self._DF_obj.get_DF(self._CF_df_floot_side.iloc[i]['end_date'], 'cubic', 'cubic')
            forward_libor = (DF_start_date / DF_end_date - 1) / dcf_list[i]
            forward_libor_list.append(forward_libor)
        return forward_libor_list
    
    def gool_seek(self):
        annuity = self.calc_pv_fix_side() / self._fixed_rate
        pv_floot_side= self.calc_pv_floot_side()
        swap_rate = pv_floot_side / annuity
        return swap_rate
    
    def calc_pv(self):
        if (self._boolean_fixed_side == True):
            sign = 1
        else:
            sign = -1
        return sign * (self.calc_pv_floot_side() - self.calc_pv_fix_side())
    
    def output_CF_to_csv(self):
        self._CF_df_floot_side.columns = ['fixing_date(floot)', 'start_date(floot)', 'end_date(floot)']
        self._CF_df_fixed_side.columns = ['fixing_date(fixed)', 'start_date(fixed)', 'end_date(fixed)']
        len_CF_floot_side = len(self._CF_df_floot_side)
        self._CF_df_floot_side['fixing_rate'] = [float(0) for i in range(len_CF_floot_side)]
        self._CF_df_fixed_side['fixed_rate']= self._fixed_rate
        if (self._boolean_fixed_side):
            df_concat = pd.concat([self._CF_df_fixed_side, self._CF_df_floot_side], axis=1)
        else:
            df_concat = pd.concat([self._CF_df_floot_side, self._CF_df_fixed_side], axis=1)
        df_concat.to_csv('CF.csv')

In [1020]:
base_date = '2018/9/28'
start_date = '2018/8/10'
end_date = '2028/8/10'
fixed_rate = 0.030052448128125624
notional = 1
CCY = 'USD'
freq_fixed_side = '6M'
freq_floot_side = '3M'
str_dcc_fixed_side = '360'
str_dcc_floot_side = '360'
is_fixed_pay = True
df_IR = df_IR_180928

In [1021]:
test_trade_obj = swap_pricer(base_date, start_date, end_date, fixed_rate, notional, CCY, freq_fixed_side, freq_floot_side, str_dcc_fixed_side,str_dcc_floot_side, is_fixed_pay, df_IR)


In [1022]:
test_trade_obj.calc_pv_floot_side()

0.2558451108436123

In [1023]:
test_trade_obj.calc_pv_fix_side()

0.24763948026241137

In [1026]:
test_trade_obj.calc_pv()

0.008205630581200912

In [1019]:
test_trade_obj.generate_forward_libor()

[0.024123370079415764,
 0.02690506095531207,
 0.02728591580713421,
 0.028748179673801333,
 0.030007848614566263,
 0.030518361885545065,
 0.030752881648882457,
 0.030764603313526998,
 0.03062515712631145,
 0.030407950027026447,
 0.03018848300075832,
 0.0300484622903157,
 0.03000246698404939,
 0.029964567530084675,
 0.029917510184204698,
 0.029880781982579468,
 0.029848858333262392,
 0.029826285217047898,
 0.02981433847820463,
 0.02982998921484768,
 0.02986408217863716,
 0.029910315195466875,
 0.029965066051445,
 0.030038430140133812,
 0.030119865204193748,
 0.030193851414667565,
 0.030250395247450403,
 0.030299785641625002,
 0.030337337839862068,
 0.030390493377276374,
 0.03046120916366634,
 0.0305649773667089,
 0.030688060825593038,
 0.03081317367120454,
 0.03092973834575167,
 0.03105075532867536,
 0.031160168277711894,
 0.031243484770913005,
 0.031291532121109356,
 0.03130970329148075]

In [909]:
pd.read_csv('CF.csv').loc[:, 'fixing_date(floot)':'fixing_rate'].loc[0]['fixing_rate']

0.03

In [752]:
base_date = '2018/11/2'
start_date = '2018/12/10'
end_date = '2028/12/10'
fixed_rate = 0.01
notional = 100000000
CCY = 'USD'
freq_fixed_side = '6M'
freq_floot_side = '3M'
str_dcc_fixed_side = '360'
str_dcc_floot_side = '360'
is_fixed_pay = True

In [613]:
IRS_obj = swap_pricer(base_date, start_date, end_date, par_rate, notional, CCY, freq_fixed_side, freq_floot_side, str_dcc_fixed_side,str_dcc_floot_side, is_fixed_pay, df_IR)
IRS_obj.calc_pv()

-61218.49067253247

In [ ]:
pd.DataFrame.

### 2018/11/28
- 問題：セータが出てこないので，要プログラム変更
- 方針：キャッシュフローの部分をbase_dateからgenerateしないといけないはず

## test space

In [11]:
swap_pricer()

In [780]:
import datetime

def get_dtobj_int_year_month_day(str_date):
    dt_obj_date = datetime.datetime.strptime(str_date, '%Y/%m/%d')
    int_year = dt_obj_date.year
    int_month = dt_obj_date.month
    int_day = dt_obj_date.day
    return dt_obj_date, int_year, int_month, int_day

def generate_list_CF(str_start_date, str_end_date, str_frequency):
    int_frequency = int(str_frequency[:len(str_frequency)-1])
    dt_obj_start_date, int_year, int_month, int_day = get_dtobj_int_year_month_day(str_start_date)
    dt_obj_end_date = get_dtobj_int_year_month_day(str_end_date)[0]
    dt_obj_CF = dt_obj_start_date
    CF_array = [dt_obj_CF.strftime('%Y/%m/%d')]
    while dt_obj_CF < dt_obj_end_date:
        int_month = int_month + int_frequency
        if int_month > 12:
            int_month = int_month - 12
            int_year = int_year + 1
            str_CF = str(int_year) + '/' + '{0:02d}'.format(int_month) + '/' + '{0:02d}'.format(int_day)
            CF_array.append(str_CF)
            dt_obj_CF = datetime.datetime.strptime(str_CF, '%Y/%m/%d')
        else:
            str_CF = str(int_year) + '/' + '{0:02d}'.format(int_month) + '/' + '{0:02d}'.format(int_day)
            CF_array.append(str_CF)
            dt_obj_CF = datetime.datetime.strptime(str_CF, '%Y/%m/%d')
    return CF_array

def generate_list_CF_with_days(str_start_date, str_end_date, str_frequency):
    CF_list = generate_list_CF(str_start_date, str_end_date, str_frequency)
    len_CF_list = len(CF_list)
    dt_obj_start_date = datetime.datetime.strptime(CF_list[0], '%Y/%m/%d')
    days_list = []
    for i in range(len_CF_list):
        dt_obj_target_date = datetime.datetime.strptime(CF_list[i], '%Y/%m/%d')
        days = (dt_obj_target_date - dt_obj_start_date).days
        days_list.append(days)
    df_CF = pd.DataFrame(CF_list)
    df_days = pd.DataFrame(days_list)
    df_CF_with_days = pd.concat([df_days, df_CF], axis=1)
    return df_CF_with_days.values.tolist()

def generate_CF_df(str_start_date, str_end_date, str_frequency, is_floot):
    '''
    generate Cashflow with fixing date, start date and end date. 
    The output form is dataframe object after concatinating.
    '''
    original_CF_list = generate_list_CF(str_start_date, str_end_date, str_frequency)
    start_CF_list = original_CF_list[:-1]
    end_CF_list = original_CF_list[1:]
    fixing_CF_list = []
    if (is_floot):
        for i in range(len(start_CF_list)):
            fixing_CF = (str_to_dt_obj(start_CF_list[i]) - datetime.timedelta(days=2)).strftime('%Y/%m/%d')
            fixing_CF_list.append(fixing_CF)
    else:
        fixing_CF_list = ['1900/01/01' for i in range(len(start_CF_list))]
    df_start_CF = pd.DataFrame(start_CF_list)
    df_end_CF = pd.DataFrame(end_CF_list)
    df_fixing_CF = pd.DataFrame(fixing_CF_list) 
    df_concat = pd.concat([df_fixing_CF, df_start_CF, df_end_CF], axis=1)
    df_concat.columns = ['fixing_date', 'start_date', 'end_date']
    return df_concat

def get_CF_df_from_base_date(str_base_date, str_start_date, str_end_date, str_frequency, is_floot):
    original_df_CF = generate_CF_df(str_start_date, str_end_date, str_frequency, is_floot)
    dt_obj_base_date = str_to_dt_obj(str_base_date)
    dt_obj_start_date = str_to_dt_obj(str_start_date)
    for i in range(len(original_df_CF)):
        dt_obj_target_date = str_to_dt_obj(original_df_CF ['start_date'][i])
        diff =  (dt_obj_base_date - dt_obj_target_date).days
        index_num = i
        if diff <= 0:
            break
    new_df_CF = original_df_CF[max([index_num, 0]):]
#    if dt_obj_base_date > dt_obj_start_date:
 #       new_df_CF.loc[max([index_num - 1, 0])]['start_date'] = dt_obj_base_date.strftime('%Y/%m/%d')
    return new_df_CF, index_num
                                          
def get_list_CF_from_base_date(str_base_date, str_start_date, str_end_date, str_frequency):
    original_CF_list = generate_list_CF_with_days(str_start_date, str_end_date, str_frequency)
    dt_obj_start_date = datetime.datetime.strptime(original_CF_list[0][1], '%Y/%m/%d')
    dt_obj_base_date = datetime.datetime.strptime(str_base_date, '%Y/%m/%d')
    for index, CF_list  in enumerate(original_CF_list):
        dt_obj_target_date = datetime.datetime.strptime(CF_list[1], '%Y/%m/%d')
        diff =  (dt_obj_base_date - dt_obj_target_date).days
        if diff <= 0:
            break
    new_CF_list = original_CF_list[index:]
    return [j for i, j in new_CF_list]

def str_to_dt_obj(str_date):
    dt_obj = datetime.datetime.strptime(str_date, '%Y/%m/%d')
    return dt_obj

In [508]:
get_CF_df_from_base_date('2019/11/2', '2018/9/10', '2028/9/10', '6M', is_floot=True)

,fixing_date,start_date,end_date
2,2019/09/08,2019/09/10,2020/03/10
3,2020/03/08,2020/03/10,2020/09/10
4,2020/09/08,2020/09/10,2021/03/10
5,2021/03/08,2021/03/10,2021/09/10
6,2021/09/08,2021/09/10,2022/03/10
7,2022/03/08,2022/03/10,2022/09/10
8,2022/09/08,2022/09/10,2023/03/10
9,2023/03/08,2023/03/10,2023/09/10
10,2023/09/08,2023/09/10,2024/03/10
11,2024/03/08,2024/03/10,2024/09/10


In [12]:
get_list_CF_from_base_date('2018/8/2', '2018/8/10', '2028/8/10', '6M')

['2018/08/10',
 '2019/02/10',
 '2019/08/10',
 '2020/02/10',
 '2020/08/10',
 '2021/02/10',
 '2021/08/10',
 '2022/02/10',
 '2022/08/10',
 '2023/02/10',
 '2023/08/10',
 '2024/02/10',
 '2024/08/10',
 '2025/02/10',
 '2025/08/10',
 '2026/02/10',
 '2026/08/10',
 '2027/02/10',
 '2027/08/10',
 '2028/02/10',
 '2028/08/10']

In [85]:
start_date = datetime.datetime.strptime('2018/10/10', "%Y/%m/%d")
end_date = datetime.datetime.strptime('2018/11/10', "%Y/%m/%d")

In [868]:
def test_func(input_num):
    num = input_num
    return pd.DataFrame([[0,1],[2,3]]), num